In [10]:
import pickle
import pandas as pd
import numpy as np
import torch
from htgtm_model import HTGTM,GNNNet,TransformerBlock
from pytorch_tabnet import tab_network
from torch_geometric.utils import from_networkx
from torch.nn.utils import clip_grad_norm_

# process each data modality (may take some time)

In [9]:
!python ./htgtm_prepare_tabular_data.py
!python ./htgtm_prepare_graph_data.py
!python ./htgtm_prepare_time-series_data.py

finished processing time-series data


# load all data

In [13]:
user_id_remapping = pickle.load(open(r'.\user_id_remapping.pkl','rb'))

tabular_data_train = pd.read_csv(r'.\temp\processed_tabular_data_train.csv',index_col = 0).sort_index()
tabular_data_test = pd.read_csv(r'.\temp\processed_tabular_data_test.csv',index_col = 0).sort_index()

data_cols = tabular_data_train.columns

tabular_data = pd.concat([tabular_data_train, tabular_data_test], axis=0)

tabular_data.index = tabular_data.index.map(lambda x: user_id_remapping[x])

tabular_data_xgboost = tabular_data.copy()

tabular_data = tabular_data.values

tabular_data = (tabular_data - tabular_data.mean(axis=0)) / (tabular_data.std(axis=0) + 1e-8)

with open(r'.\temp\G.pkl','rb') as f:
    G = pickle.load(f)

graphdata = from_networkx(G)

tsdatavalues_full = np.load(r'.\temp\datavalues_full_std.npy')
tstime_stamps_full = np.load(r'.\temp\time_stamps_full_std.npy')
tsmasks_full = np.load(r'.\temp\masks_full_std.npy')

train_index = pickle.load(open(r'.\train_index.pkl','rb'))
val_index = pickle.load(open(r'.\val_index.pkl','rb'))

target = pd.read_csv(r'.\retention_gt_train.csv',index_col = 0).sort_index()
target.index = target.index.map(lambda x: user_id_remapping[x])

# data reweighting

In [14]:
from scipy.stats import pareto
import numpy as np

# fit the data to a Pareto distribution
params = pareto.fit(target)

pdf_fitted = pareto.pdf(target, *params)

c:\Users\sdyy6\anaconda3\envs\py39\lib\site-packages\scipy\stats\_distn_infrastructure.py:2640: RuntimeWarning: invalid value encountered in multiply
  Lhat = muhat - Shat*mu


# model hyper-parameters

In [23]:
n_d = 8
n_a = 8
n_steps = 3
gamma = 1.3
cat_idxs = []
cat_dims = [] 
cat_emb_dims = [] 
n_independent = 2
n_shared = 2
epsilon = 1e-15
momentum = 0.02
lambda_sparse = 1e-3
seed = 0
clip_value = 1
verbose = 1
optimizer_fn = torch.optim.Adam
optimizer_params = dict(lr=2e-2)
mask_type = "sparsemax"
input_dim = None
output_dim = None
device_name = "auto"
n_shared_decoder = 1
n_indep_decoder = 1
patience=10
virtual_batch_size = 128

gnnmodel = GNNNet(in_channels=tabular_data.shape[1], embedding_dim=32, out_channels=16, dropout_rate=0.5, number_of_layers=3)

tsmodel = TransformerBlock()

tabmodel = tab_network.TabNet(tabular_data.shape[1],
                           16,
                            n_d=n_d,
                            n_a=n_a,
                            n_steps=n_steps,
                            gamma=gamma,
                            cat_idxs=cat_idxs,
                            cat_dims=cat_dims,
                            cat_emb_dim=cat_emb_dims,
                            n_independent=n_independent,
                            n_shared=n_shared,
                            epsilon=epsilon,
                            virtual_batch_size=virtual_batch_size,
                            momentum=momentum,
                            mask_type=mask_type)

model = HTGTM(gnnmodel, tsmodel, tabmodel).cuda()

In [24]:
# create a simple batch_index loader with shuffle = True
dataloader_train = torch.utils.data.DataLoader(np.concatenate([
                                                                train_index
                                                                ]), batch_size=64, shuffle=True)
dataloader_total = torch.utils.data.DataLoader(range(
                                                     tabular_data_xgboost.shape[0]
                                                        ), batch_size=42, shuffle=False)

In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# define a weighted mse loss
def weighted_mse_loss(input, target, weight):
    return torch.mean(weight * (input - target) ** 2)

loss_fn = weighted_mse_loss

# train the model on training set

In [26]:
for epoch in range(10):

    if epoch < 5:

        loss_weights = pd.DataFrame(np.log1p(10000+1/pdf_fitted)/(np.log1p(10000+1/pdf_fitted).mean()), index = target.index,columns = ['weight'])

    else:

        loss_weights = pd.DataFrame(np.log1p(100+1/pdf_fitted)/(np.log1p(100+1/pdf_fitted).mean()), index = target.index,columns = ['weight'])


    model.train()
    for batch_index_train in dataloader_train:

        batch_index_train = batch_index_train.long()

        optimizer.zero_grad(set_to_none=True)

        features = torch.tensor(tabular_data[batch_index_train,:]).float().cuda()

        node_ts_feature = torch.tensor(tsdatavalues_full[batch_index_train,:,:]).view(-1,445,4).float().cuda()
        tstimestamp = torch.tensor(tstime_stamps_full[batch_index_train,:,:]).long().view(-1,445).cuda()
        tsmask = torch.tensor(tsmasks_full[batch_index_train,:,:]).bool().view(-1,445).cuda()

        node_features = torch.tensor(tabular_data).float().cuda()
        edge_index = graphdata.edge_index.long().cuda()
        edge_attr = graphdata.weight.float().cuda()

        gt = torch.tensor(target.values[batch_index_train]).float().cuda()
        sample_weight = torch.tensor(loss_weights.values[batch_index_train]).float().cuda()

        output, M_loss = model(features, node_features, edge_index, edge_attr, batch_index_train, node_ts_feature, tstimestamp, tsmask)

        loss = loss_fn(output, gt, sample_weight) - lambda_sparse * M_loss

        loss.backward()

        optimizer.step()

        clip_grad_norm_(model.parameters(), 1)

    print('training finished, epoch: ', epoch)


training finished, epoch:  0
training finished, epoch:  1
training finished, epoch:  2
training finished, epoch:  3
training finished, epoch:  4
training finished, epoch:  5
training finished, epoch:  6
training finished, epoch:  7
training finished, epoch:  8
training finished, epoch:  9


# make predictions to all data points

In [27]:
pred_nn = []

for batch_index_total in dataloader_total:

    with torch.no_grad():

        batch_index_total = batch_index_total.long()

        features = torch.tensor(tabular_data[batch_index_total,:]).view(-1,17).float().cuda()

        node_ts_feature = torch.tensor(tsdatavalues_full[batch_index_total,:,:]).view(-1,445,4).float().cuda()
        tstimestamp = torch.tensor(tstime_stamps_full[batch_index_total,:,:]).long().view(-1,445).cuda()
        tsmask = torch.tensor(tsmasks_full[batch_index_total,:,:]).bool().view(-1,445).cuda()

        node_features = torch.tensor(tabular_data).float().cuda()
        edge_index = graphdata.edge_index.long().cuda()
        edge_attr = graphdata.weight.float().cuda()

        output, M_loss = model(features, node_features, edge_index, edge_attr, batch_index_total, node_ts_feature, tstimestamp, tsmask)

        #clip output to larger than 0
        output = torch.clamp(output, min=0)

        pred_nn.append(output.cpu().numpy())

pred_nn = np.concatenate(pred_nn).reshape(-1,1)

# tune a xgboost regressor for ensemble

In [30]:
import xgboost as xgb

from sklearn.model_selection import RandomizedSearchCV

xtrain = np.concatenate([tabular_data_xgboost.values[train_index,:], pred_nn[train_index]], axis=1)
ytrain = target.values[train_index]

# subsample the training set to speed up the random search
sample_indices = np.random.choice(xtrain.shape[0], size=10000, replace=False)
xtrain_sampled = xtrain[sample_indices]
ytrain_sampled = ytrain[sample_indices]

xval = np.concatenate([tabular_data_xgboost.values[val_index,:], pred_nn[val_index]], axis=1)
yval = target.values[val_index]

# use xgb.XGBRegressor as the model
xgbmodel = xgb.XGBRegressor()

# define the hyperparameter space
param_dist = {
                'n_estimators': [100,200,300,400,500,1000],
                'learning_rate': [0.01,0.05,0.1,0.2,0.3],
                'max_depth': [3,4,5,6,7,8,9,10],
                'min_child_weight': [1,2,3,4,5,6,7,8,9,10],
                'gamma': [0,0.1,0.2,0.3,0.4,0.5],
                'subsample': [0.5,0.6,0.7,0.8,0.9,1],
                'colsample_bytree': [0.5,0.6,0.7,0.8,0.9,1],
                'reg_alpha': [0,0.1,0.2,0.3,0.4,0.5],
                'reg_lambda': [0,0.1,0.2,0.3,0.4,0.5]
                }

# run randomized search
n_iter_search = 100
random_search = RandomizedSearchCV(xgbmodel, param_distributions=param_dist,
                                      n_iter=n_iter_search, cv=3, verbose=0, n_jobs=-1)

random_search.fit(xtrain_sampled, ytrain_sampled)

# get the best model
best_model = random_search.best_estimator_

xgbmodel = xgb.XGBRegressor(**best_model.get_params())

xgbmodel.fit(xtrain, ytrain)

pred_val = xgbmodel.predict(xval)

# clip pred_val to larger than 0
pred_val = np.clip(pred_val, a_min=0, a_max=None)

# RMSE on validation set

In [ ]:
from sklearn.metrics import mean_squared_error
print('xgb val rmse: ', np.sqrt(mean_squared_error(yval, pred_val)))